# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [8]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'G  B)EKDTSTC EQ/NALLSALGMNARSEO(A', 'calculationPrice': 'tops', 'open': None, 'openTime': None, 'openSource': 'ffilcaoi', 'close': None, 'closeTime': None, 'closeSource': 'cafioifl', 'high': None, 'highTime': 1667048629652, 'highSource': 'recai merp EXl tieI', 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 135.68, 'latestSource': 'IEX real time price', 'latestTime': '12:27:02 PM', 'latestUpdate': 1676361857823, 'latestVolume': None, 'iexRealtimePrice': 136.29, 'iexRealtimeSize': 4, 'iexLastUpdated': 1682218945676, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 136.55, 'previousVolume': 177715084, 'change': 2.67, 'changePercent': 0.02019, 'volume': None, 'iexMarketPercent': 0.013656149350389443, 'iexVolume': 794061, 'avgTota

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [12]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

36.86

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [15]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [22]:
final_df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['peRatio'],
            'N/A'
        ],
        index = my_columns
        ),
            ignore_index=True
        )
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,121.710,55.12,N/A
1,AAL,17.022,-1.2,N/A
2,AAP,156.680,22.7,N/A
3,AAPL,141.000,37.76,N/A
4,ABBV,104.760,23.3,N/A
5,ABC,108.250,-6.56,N/A
6,ABMD,354.780,78.47,N/A
7,ABT,124.670,66.45,N/A
8,ACN,248.120,31.07,N/A
9,ADBE,476.500,44.86,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [29]:
final_df.sort_values('Price-to-Earnings Ratio', ascending=False, inplace=True)
final_df = final_df[final_df['Price-to-Earnings Ratio']>0]
final_df = final_df[:50]
final_df.reset_index(drop=True, inplace=True)
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,TEL,123.025,373.81,N/A
1,KMI,14.250,295.99,N/A
2,MAR,119.915,222.06,N/A
3,IQV,185.445,206.1,N/A
4,CMG,1539.500,183.44,N/A
5,SBUX,101.000,182.86,N/A
6,REG,47.630,174.48,N/A
7,NOW,565.320,171.19,N/A
8,GLW,36.740,164.11,N/A
9,DXCM,390.210,160.4,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [30]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [31]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [36]:
position_size = float(portfolio_size) / len(final_df.index)
for row in final_df.index:
    final_df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[row, 'Price'])
    
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,TEL,123.025,373.81,162
1,KMI,14.250,295.99,1403
2,MAR,119.915,222.06,166
3,IQV,185.445,206.1,107
4,CMG,1539.500,183.44,12
5,SBUX,101.000,182.86,198
6,REG,47.630,174.48,419
7,NOW,565.320,171.19,35
8,GLW,36.740,164.11,544
9,DXCM,390.210,160.4,51


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [55]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()


#Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

#Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

#Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
enterprise_ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = (enterprise_value/enterprise_ebitda)

#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = (enterprise_value/gross_profit)

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [59]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        enterprise_ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/enterprise_ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
        pd.Series(
        [
    symbol,
    data[symbol]['quote']['latestPrice'],
    'N/A',
    data[symbol]['quote']['peRatio'],
    'N/A',
    data[symbol]['advanced-stats']['priceToBook'],
    'N/A',
    data[symbol]['advanced-stats']['priceToSales'],
    'N/A',
    ev_to_ebitda,
    'N/A',
    ev_to_gross_profit,
    'N/A',
    'N/A'     
        ],
            index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [60]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,123.195,N/A,53.17,N/A,7.63,N/A,7.04,N/A,32.185778,N/A,12.916135,N/A,N/A
1,AAL,17.478,N/A,-1.26,N/A,-1.92,N/A,0.4407,N/A,-5.936555,N/A,1.359599,N/A,N/A
2,AAP,156.500,N/A,21.97,N/A,2.81,N/A,1.08,N/A,10.473057,N/A,2.364789,N/A,N/A
3,AAPL,138.920,N/A,37.63,N/A,33.7,N/A,7.72,N/A,26.188862,N/A,19.129583,N/A,N/A
4,ABBV,106.740,N/A,23.54,N/A,12.4,N/A,4.53,N/A,15.894825,N/A,9.489919,N/A,N/A
5,ABC,111.000,N/A,-6.73,N/A,-21.51,N/A,0.1162,N/A,8.797647,N/A,4.013551,N/A,N/A
6,ABMD,348.500,N/A,78.12,N/A,13.84,N/A,19.7,N/A,64.014216,N/A,23.734933,N/A,N/A
7,ABT,125.080,N/A,65.08,N/A,7.03,N/A,7.1,N/A,36.999611,N/A,13.008471,N/A,N/A
8,ACN,254.130,N/A,31.65,N/A,8.99,N/A,3.72,N/A,18.417538,N/A,11.099174,N/A,N/A
9,ADBE,492.980,N/A,45.59,N/A,16.88,N/A,17.3,N/A,48.939576,N/A,19.520354,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [63]:
rv_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

In [66]:
for column in ['Price-to-Earnings Ratio','Price-to-Book Ratio',
       'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(value=rv_dataframe[column].mean(), inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,123.195,N/A,53.170000,N/A,7.630000,N/A,7.040000,N/A,32.185778,N/A,12.916135,N/A,N/A
1,AAL,17.478,N/A,-1.260000,N/A,-1.920000,N/A,0.440700,N/A,-5.936555,N/A,1.359599,N/A,N/A
2,AAP,156.500,N/A,21.970000,N/A,2.810000,N/A,1.080000,N/A,10.473057,N/A,2.364789,N/A,N/A
3,AAPL,138.920,N/A,37.630000,N/A,33.700000,N/A,7.720000,N/A,26.188862,N/A,19.129583,N/A,N/A
4,ABBV,106.740,N/A,23.540000,N/A,12.400000,N/A,4.530000,N/A,15.894825,N/A,9.489919,N/A,N/A
5,ABC,111.000,N/A,-6.730000,N/A,-21.510000,N/A,0.116200,N/A,8.797647,N/A,4.013551,N/A,N/A
6,ABMD,348.500,N/A,78.120000,N/A,13.840000,N/A,19.700000,N/A,64.014216,N/A,23.734933,N/A,N/A
7,ABT,125.080,N/A,65.080000,N/A,7.030000,N/A,7.100000,N/A,36.999611,N/A,13.008471,N/A,N/A
8,ACN,254.130,N/A,31.650000,N/A,8.990000,N/A,3.720000,N/A,18.417538,N/A,11.099174,N/A,N/A
9,ADBE,492.980,N/A,45.590000,N/A,16.880000,N/A,17.300000,N/A,48.939576,N/A,19.520354,N/A,N/A


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [110]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric]) / 100
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,38.358,521,-6.60,0.38,0.5160,0.28,0.7309,0.4,4.341656,0.24,0.687603,0.14,5.90099
1,AAL,17.478,1144,-1.26,0.46,-1.9200,0.18,0.4407,0.24,-5.936555,0.12,1.359599,0.4,6.29703
2,UNM,24.180,827,5.28,0.56,0.4493,0.24,0.4059,0.2,2.504165,0.18,0.395212,0.04,6.33663
3,F,11.395,1755,-276.03,0.08,1.2800,0.68,0.3232,0.08,4.511756,0.26,2.219464,0.66,7.08911
4,MET,49.130,407,8.12,0.62,0.6030,0.32,0.6896,0.38,4.542433,0.28,0.656613,0.12,7.64356
5,BA,197.220,101,-25.58,0.2,-9.6600,0.12,1.9000,0.86,-26.461918,0.04,-188.656944,0.02,8.81188
6,FTI,10.828,1847,-0.89,0.5,1.1800,0.58,0.3777,0.16,3.235676,0.2,2.132064,0.64,9.08911
7,LNC,46.710,428,11.52,0.74,0.4130,0.22,0.4991,0.26,8.213478,0.68,0.485063,0.06,9.30693
8,HIG,50.700,394,10.29,0.68,0.9927,0.46,0.8411,0.48,4.211657,0.22,0.860033,0.2,9.34653
9,ALL,109.400,182,7.72,0.6,1.3300,0.7,0.7385,0.42,2.446290,0.16,0.718402,0.16,9.78218


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [111]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row,'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,38.358,521,-6.60,0.38,0.5160,0.28,0.7309,0.4,4.341656,0.24,0.687603,0.14,0.288
1,AAL,17.478,1144,-1.26,0.46,-1.9200,0.18,0.4407,0.24,-5.936555,0.12,1.359599,0.4,0.28
2,UNM,24.180,827,5.28,0.56,0.4493,0.24,0.4059,0.2,2.504165,0.18,0.395212,0.04,0.244
3,F,11.395,1755,-276.03,0.08,1.2800,0.68,0.3232,0.08,4.511756,0.26,2.219464,0.66,0.352
4,MET,49.130,407,8.12,0.62,0.6030,0.32,0.6896,0.38,4.542433,0.28,0.656613,0.12,0.344
5,BA,197.220,101,-25.58,0.2,-9.6600,0.12,1.9000,0.86,-26.461918,0.04,-188.656944,0.02,0.248
6,FTI,10.828,1847,-0.89,0.5,1.1800,0.58,0.3777,0.16,3.235676,0.2,2.132064,0.64,0.416
7,LNC,46.710,428,11.52,0.74,0.4130,0.22,0.4991,0.26,8.213478,0.68,0.485063,0.06,0.392
8,HIG,50.700,394,10.29,0.68,0.9927,0.46,0.8411,0.48,4.211657,0.22,0.860033,0.2,0.408
9,ALL,109.400,182,7.72,0.6,1.3300,0.7,0.7385,0.42,2.446290,0.16,0.718402,0.16,0.408


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [112]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,24.180,827,5.28,0.56,0.4493,0.24,0.4059,0.2,2.504165,0.18,0.395212,0.04,0.244
1,BA,197.220,101,-25.58,0.2,-9.6600,0.12,1.9000,0.86,-26.461918,0.04,-188.656944,0.02,0.248
2,AAL,17.478,1144,-1.26,0.46,-1.9200,0.18,0.4407,0.24,-5.936555,0.12,1.359599,0.4,0.28
3,AIG,38.358,521,-6.60,0.38,0.5160,0.28,0.7309,0.4,4.341656,0.24,0.687603,0.14,0.288
4,PRU,82.750,241,-302.43,0.06,0.4804,0.26,0.5204,0.28,22.323486,1,0.511141,0.08,0.336
5,MET,49.130,407,8.12,0.62,0.6030,0.32,0.6896,0.38,4.542433,0.28,0.656613,0.12,0.344
6,F,11.395,1755,-276.03,0.08,1.2800,0.68,0.3232,0.08,4.511756,0.26,2.219464,0.66,0.352
7,LYV,70.920,282,-10.05,0.32,-147.4000,0.02,3.2000,1,-19.091803,0.06,1.779439,0.56,0.392
8,LNC,46.710,428,11.52,0.74,0.4130,0.22,0.4991,0.26,8.213478,0.68,0.485063,0.06,0.392
9,ALL,109.400,182,7.72,0.6,1.3300,0.7,0.7385,0.42,2.446290,0.16,0.718402,0.16,0.408


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [114]:
portfolio_input()

Enter the value of your portfolio:100000


In [115]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[row, 'Price'])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,24.180,82,5.28,0.56,0.4493,0.24,0.4059,0.2,2.504165,0.18,0.395212,0.04,0.244
1,BA,197.220,10,-25.58,0.2,-9.6600,0.12,1.9000,0.86,-26.461918,0.04,-188.656944,0.02,0.248
2,AAL,17.478,114,-1.26,0.46,-1.9200,0.18,0.4407,0.24,-5.936555,0.12,1.359599,0.4,0.28
3,AIG,38.358,52,-6.60,0.38,0.5160,0.28,0.7309,0.4,4.341656,0.24,0.687603,0.14,0.288
4,PRU,82.750,24,-302.43,0.06,0.4804,0.26,0.5204,0.28,22.323486,1,0.511141,0.08,0.336
5,MET,49.130,40,8.12,0.62,0.6030,0.32,0.6896,0.38,4.542433,0.28,0.656613,0.12,0.344
6,F,11.395,175,-276.03,0.08,1.2800,0.68,0.3232,0.08,4.511756,0.26,2.219464,0.66,0.352
7,LYV,70.920,28,-10.05,0.32,-147.4000,0.02,3.2000,1,-19.091803,0.06,1.779439,0.56,0.392
8,LNC,46.710,42,11.52,0.74,0.4130,0.22,0.4991,0.26,8.213478,0.68,0.485063,0.06,0.392
9,ALL,109.400,18,7.72,0.6,1.3300,0.7,0.7385,0.42,2.446290,0.16,0.718402,0.16,0.408


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [119]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')

rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [120]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [121]:
column_formats = {

        'A': ['Ticker',string_template], 
        'B': ['Price', dollar_template],
        'C': ['Number of Shares to Buy', integer_template],
        'D': ['Price-to-Earnings Ratio', float_template],
        'E': ['PE Percentile', percent_template],
        'F': ['Price-to-Book Ratio', float_template],
        'G': ['PB Percentile', percent_template],
        'H': ['Price-to-Sales Ratio', float_template],
        'I': ['PS Percentile', percent_template],
        'J': ['EV/EBITDA', float_template],
        'K': ['EV/EBITDA Percentile', percent_template],
        'L': ['EV/GP', float_template],
        'M': ['EV/GP Percentile', percent_template],
        'N': ['RV Score', percent_template]
    }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [25]:
writer.save()